In [2]:
import tweepy
import time
import logging
import random
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import os
from os import environ
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()
# Authenticating to to Twitter
API_KEY = "3ZNV1tHj4j3WN6JKjnriPthYR"
API_SKEY = "ZOHxNBrfpEMnrOcyNd5Eb2rQb0OyWUL0JoO9CwB1VCy0A0vfca"
ACC_Token = "1353704693910921221-6VLjs0f74mvtLrETgX2CuXJ6EVgHan"
ACC_Token_Secret = "hPeJcfv9pFb5un5AHiFpDKfs4fhRw2OQ7emjgc0QG2jVw"
auth = tweepy.OAuthHandler(API_KEY,API_SKEY)
auth.set_access_token(ACC_Token,ACC_Token_Secret)
api = tweepy.API(auth, wait_on_rate_limit=True,
    wait_on_rate_limit_notify=True)
#Credential for firebase
cred = credentials.Certificate("twitter-threader-firebase-adminsdk-rh7d4-60df64a83c.json")

class userThread:
    def __init__(self,id, name,username,profile_img,tweets):
        '''
        id: profile id of thread owner
        '''
        self.id = id
        self.name = name
        self.username = username
        self.tweets = tweets
        self.profile_img = profile_img
    def to_dict(self):
        obj = {
            'id':self.id,
            'name':self.name,
            'username':self.username,
            'profile_img':self.profile_img,
            'tweets':[]
        }
        for tweet in self.tweets:
            tweet_id = tweet.tweet_id
            obj['tweets'].append(tweet.to_dict())
        return obj
class Tweet:
    def __init__(self, text,date,medias,tweet_id,urls):
        self.text = text
        self.date = date
        self.medias = medias
        self.tweet_id = tweet_id
        self.urls = urls ##contains list either empty or list with object url,expanded_url,display_url(sliced urls)
    def to_dict(self):
        obj = {
            'text':self.text,
            'date':self.date,
            'tweet_id':str(self.tweet_id),
            'urls':self.urls,
            'medias':self.medias
        }
        return obj
class ThreadCompiler:
    def __init__(self,tweet_id,user_id,thread_request_id,easy_compile=True,max_tweets_to_look=500):
        '''
        tweet_id - the id of the tweet under which user requested
        user_id - id of the thread owner
        thread_request_id - id of the user who requested thread compile
        easy_compile - rolling up to the parent (use when thread is too old)
        '''
        self.tweet_id = tweet_id
        self.id = tweet_id ##storing parent id
        self.user_id = user_id #thread_user_ids
        self.tweets = None
        self.max_tweets_to_look = max_tweets_to_look
        self.thread_request_id = thread_request_id
        self.easy_compile = easy_compile #if easy compile we won't be checking Bottom thread just roll to the parent of thread
    def compileTweets(self):
        '''
        Compiles tweet of thread
        Compile to the parent of the child
        return list of object of class type Tweet
        '''
        self.tweets = []
        parent_tweet_id = self.tweet_id
        print("ThreadCompiler: Fetching Head thread!")
        while parent_tweet_id:
            try:
                tweet = api.get_status(parent_tweet_id, tweet_mode="extended")
                if tweet.user.id != self.user_id:
                    break
                medias = []
                if hasattr(tweet,"extended_entities"):
                    if "media" in tweet.extended_entities:
                        media_entities = tweet.extended_entities['media']
                        for media_data in media_entities:
                            media = {}
                            ##Currently only planning to save photo data as it'll be transformed to PDF
                            if media_data['type']=='photo' or media_data['type'] == 'video':
                                media['type'] = media_data['type']
                                media['media_url_https'] = media_data['media_url_https']
                                media['media_url'] = media_data['media_url']
                                media['expanded_url'] = media_data['expanded_url']
                                medias.append(media)
                tweetObj = Tweet(tweet.full_text,str(tweet.created_at),list(medias),parent_tweet_id,tweet.entities['urls'])
                self.tweets.append(tweetObj)
                parent_tweet_id = tweet.in_reply_to_status_id
            except tweepy.TweepError as e:
                print("ThreadCompiler:TweepyError {}".format(e))
                #Send Response to user and end the compilation
                raise Exception("Compiling thread top error!")
                break
        self.tweets = self.tweets[::-1]
        if self.easy_compile:
            print("ThreadCompiler:Easy Compile Requested!")
            return self.tweets
        else:
            return self.tweetCompilerBottom()
        #return list(self.tweets)
    def tweetCompilerBottom(self,since_id=None):
        '''
        Compiles the bottom half of the tweet by looking for child
        '''
        print("ThreadCompiler:Fetching Bottom thread!")
        if not since_id:
            looking_for = self.tweet_id
        tweets = tweepy.Cursor(api.user_timeline,user_id=self.user_id,since_id=looking_for,trim_user=True,include_rts=False,               exclude_replies=False,tweet_mode="extended").items()
        tweets_track = {} ##contains all tweets traced
        bottom_thread_exists = False
        for i in range(self.max_tweets_to_look):
            try:
                tweet = tweets.next()
                if hasattr(tweet, 'in_reply_to_status_id_str') and tweet.in_reply_to_status_id:
                    medias = []
                    if hasattr(tweet,"extended_entities"):
                        if "media" in tweet.extended_entities:
                            media_entities = tweet.extended_entities['media']
                            for media_data in media_entities:
                                media = {}
                                ##Currently only planning to save photo data as it'll be transformed to PDF
                                if media_data['type']=='photo' or media_data['type'] == 'video':
                                    media['type'] = media_data['type']
                                    media['media_url_https'] = media_data['media_url_https']
                                    media['media_url'] = media_data['media_url']
                                    media['expanded_url'] = media_data['expanded_url']
                                    medias.append(media)
                    tweetObj = Tweet(tweet.full_text,str(tweet.created_at),list(medias),tweet.id,tweet.entities['urls'])
                    #Saving track with respect to in_reply_to_status_id ez to look for thread
                    #print(i)
                    tweets_track[tweet.id] = {
                        "tweet":tweetObj,
                        "in_reply_to_status_id":tweet.in_reply_to_status_id
                    }
            except StopIteration:
                bottom_thread_exists = True #No more results might have reached to since_id and hence thread exists
                print("ThreadCompiler: BottomThread - No more result found")
                break
            except tweepy.RateLimitError as e:
                logging.error("Twitter api rate limit reached Error-{}".format(e))
                time.sleep(60) ##Sleep and retry after a while
                continue
            except tweepy.TweepError as e:
                print("ThreadCompiler:TweepyError {}".format(e))
                #Send Response to user and end the compilation
                raise Exception("Compiling bottom thread error!")
                break
        #return tweets_track
        if bottom_thread_exists:
            self.fetchBottomThread(tweets_track)
        else:
            #Respone thread is to old
            pass
        return self.tweets
    def fetchBottomThread(self,tweets_track,tweet_id=None):
        if not tweet_id:
            tweet_id = self.tweet_id
        '''
        Finding Longest Thread till tweet_id
        if multiple exists returns the on which comes first
        '''
        print("ThreadCompiler:Cooking Bottom Threads!")
        i=0
        threads = {}
        while i<len(list(tweets_track)):
            looking_for = list(tweets_track)[i]
            child_id = looking_for
            i +=1
            count = 0
            possible_threads = []
            while looking_for:
                if looking_for in tweets_track:
                    count +=1
                    possible_threads.append(tweets_track[looking_for]['tweet'])
                    looking_for = tweets_track[looking_for]["in_reply_to_status_id"]
                elif looking_for == tweet_id: #Thread I was looking for
                    threads[child_id] = possible_threads
                    break
                else:
                    break
            #i += count - 1
        if len(threads) > 0:
            thread_id = max(threads, key=lambda k: len(threads[k]))
            self.tweets += threads[thread_id][::-1]
            print("ThreadCompiler:Bottom Threads prepared!")
            return threads[thread_id][::-1]
        else:
            print("ThreadCompiler:Bottom Thread does not exists!")
            return
    def compileThread(self):
        '''
        Compiles Thread of tweets and user and return object of class type userThread
        '''
        print("ThreadCompiler: Compiling Thread")
        if not self.tweets:
            self.compileTweets()
        self.id = self.tweets[0].tweet_id ##first id is used to save the thread
        user = api.get_user(self.user_id)
        return userThread(self.user_id,user.name,user.screen_name,user.profile_image_url_https,self.tweets)
    def save(self,threaData:"Dictionar"=None):
        '''
        threaData: Takes dictionary
        saves it to the firebase if not already exists and return True
        '''
        ##
        if not threaData:
            threaData = self.compileThread().to_dict()
        print("ThreadCompiler: Preparing to save!")
        FU = FirebaseUtility(cred)
        FU.storeData(self.id,threaData)
        return True
    def getThreadID(self):
        '''
        Returns the thread id
        Used to access documents
        '''
        return str(self.id)
class FirebaseUtility:
    def __init__(self,cred):
        self.cred = cred
        self.initialize() ##initialise first
        self.db = firestore.client()
    def initialize(self):
        try:
            if not firebase_admin._apps:
                print("FirebaseUtility:Initializing")
                firebase_admin.initialize_app(self.cred)
        except:
            print("FirebaseUtility:Error in cred FIX NEEDED!")
    def documentExists(self,thread_id:"Thread parent id",thread_len:"Length of fetched thread"):
        doc_ref = self.db.collection(u'threads').document(str(thread_id))
        doc = doc = doc_ref.get()
        if doc.exists:
            if len(doc.to_dict()['tweets']) < thread_len:
                print("FirebaseUtility:Thread Already Exists but shorter")
                return False
            else:
                print("FirebaseUtility:Thread Already Exists")
                return True
        else:
            print('FirebaseUtility:No such Thread!')
            return False
    def storeData(self,thread_id,data:"dictionary"):
        '''
        Stores the dictionary data with document name thread_id
        '''
        if not self.documentExists(thread_id,len(data['tweets'])):
            doc_ref = self.db.collection(u'threads').document(str(thread_id))
            doc_ref.set(data)
            print('FirebaseUtility:Thread {} Stored!'.format(str(thread_id)))
        else:
            pass
class ThreaderBot:
    def __init__(self,file_name="since_id.txt"):
        fread = open(file_name, 'r')
        self.since_id = int(fread.read().strip())
        fread.close()
    def retrieve_since_id(self,file_name="since_id.txt"):
        fread = open(file_name, 'r')
        since_id = int(fread.read().strip())
        fread.close()
        return since_id

    def store_since_id(self,since_id, file_name="since_id.txt"):
        fwrite = open(file_name, 'w')
        fwrite.write(str(since_id))
        fwrite.close()
        return
    def fetchTweets(self):
        '''
        Fetches only mentioned tweets
        retweet will trigger this aswell
        '''
        self.since_id = self.retrieve_since_id()
        mentions = api.mentions_timeline(self.since_id)
        mention = mentions[0] if len(mentions) !=0 else None
        if mention:
            print("Storing The last mentioned",mention.id)
            since_id = mention.id #Store the last id so that we can keep ourself updated
            self.store_since_id(since_id)
            self.since_id = since_id #Update the bot aswell
        return mentions
    def run(self):
        '''
        Returns unique list of recently mentioned tweets
        in_reply_to_status_id,in_reply_to_user_id,requested user screen_name and request id
        Note:Twitter doesn't allow to tweet same tweet to same reply
        '''
        print("ThreaderBot: Running...")
        try:
            tweets = self.fetchTweets()
            if not tweets:
                print("ThreaderBot: Nothing New!")
                return False
            else:
                print("ThreaderBot: Threading...")
                request_details = []
                for tweet in tweets:
                    easy_compile = True
                    if "compile" in tweet.text.lower():
                        easy_compile = False
                    request_details.append((tweet.in_reply_to_status_id,tweet.in_reply_to_user_id,tweet.user.screen_name,tweet.id,easy_compile))
                request_details = list(set(request_details))
                return request_details
        except tweepy.TweepError as e:
                logging.error("ThreaderBot: Twitter api Error {}".format(e))
                return
    def sendResponse(self,text,request_username,rquest_id):
        '''
        Send response who requested the thread
        username is required to reply
        Note: make sure that twitter api project is created under read and write
        '''
        respone = "@"+request_username+" "+str(text)
        try:
            api.update_status(respone,rquest_id)
            print("Request Successful")
        except:
            logger.error("Error replying to the tweet", exc_info=True)
def surfBot(bot:"ThreadBot"):
    '''
    Runs the bot and make him awake
    '''
    requests = bot.run()
    if requests:
        for in_reply_to_tweet_id,in_reply_to_user_id,request_username,request_id,easy_compile in requests:
            try:
                compiler = ThreadCompiler(in_reply_to_tweet_id,in_reply_to_user_id,request_id,easy_compile)
                if compiler.save():
                    text = "Thread URL goes here and Thread Length - "+str(len(compiler.tweets))
                    bot.sendResponse(text,request_username,request_id)
                    print(compiler.getThreadID())
                else:
                    print("Bot Surfer:Nothing Requested!")
                    return
            except Exception as e:
                ##Send response as we got error
                text = "Looks Like thread is too old or something went wrong!\nTry different method check here for help - https://sobydamn.github.io/TwitterThread/"
                bot.sendResponse(text,request_username,request_id)
                print("SurfBot: Error - {}".format(e))

In [3]:
bot = ThreaderBot()

In [16]:
bot.run()

ThreaderBot: Running...
ThreaderBot: Nothing New!


False

In [4]:
surfBot(bot)

ThreaderBot: Running...
Storing The last mentioned 1356247884761542658
ThreaderBot: Threading...
ThreadCompiler: Compiling Thread
ThreadCompiler: Fetching Head thread!
ThreadCompiler:Easy Compile Requested!
ThreadCompiler: Preparing to save!
FirebaseUtility:Initializing
FirebaseUtility:No such Thread!
FirebaseUtility:Thread 1289444851868565504 Stored!
Request Successful
1289444851868565504


In [33]:
current_tweet_details = bot.run()

ThreaderBot: Running...
Storing The last mentioned 1356173077281992707
ThreaderBot: Threading...


In [34]:
in_reply_to_tweet_id,in_reply_to_user_id,request_username,request_id = current_tweet_details[0]

In [42]:
compiler = ThreadCompiler(in_reply_to_tweet_id,in_reply_to_user_id,request_id)

In [43]:
thread = compiler.compileThread()
thread.to_dict()

ThreadCompiler: Compiling Thread
ThreadCompiler: Fetching Head thread!
ThreadCompiler:Fetching Bottom thread!
ThreadCompiler: BottomThread - No more result found
ThreadCompiler:Cooking Bottom Threads!
ThreadCompiler:Bottom Threads prepared!


{'id': 722743883617214464,
 'name': 'SobyDamn',
 'username': 'SobyDamn',
 'profile_img': 'https://pbs.twimg.com/profile_images/1328960664493285378/qRrzvNSN_normal.jpg',
 'tweets': {'1355756838117986307': {'text': 'Test 3\nThread part 1',
   'date': '2021-01-31 05:56:33',
   'tweet_id': 1355756838117986307,
   'urls': [],
   'medias': []},
  '1355756893684203522': {'text': 'Test 3\nThread part 2',
   'date': '2021-01-31 05:56:46',
   'tweet_id': 1355756893684203522,
   'urls': [],
   'medias': []},
  '1355756940379361287': {'text': 'Test 3\nThread part 3',
   'date': '2021-01-31 05:56:57',
   'tweet_id': 1355756940379361287,
   'urls': [],
   'medias': []},
  '1355756980879515649': {'text': 'Test 3\nThread part 4',
   'date': '2021-01-31 05:57:07',
   'tweet_id': 1355756980879515649,
   'urls': [],
   'medias': []},
  '1356171506833829891': {'text': 'Test 3\nOOps here is thread part 5',
   'date': '2021-02-01 09:24:18',
   'tweet_id': 1356171506833829891,
   'urls': [],
   'medias': []}

In [36]:
thread = compiler.compileThread()
thread.to_dict()

ThreadCompiler: Compiling Thread
ThreadCompiler: Fetching Head thread!
ThreadCompiler:Fetching Bottom thread!
ThreadCompiler: BottomThread - No more result found
ThreadCompiler:Cooking Bottom Threads!
ThreadCompiler:Bottom Threads prepared!


{'id': 722743883617214464,
 'name': 'SobyDamn',
 'username': 'SobyDamn',
 'profile_img': 'https://pbs.twimg.com/profile_images/1328960664493285378/qRrzvNSN_normal.jpg',
 'tweets': {'1355756838117986307': {'text': 'Test 3\nThread part 1',
   'date': '2021-01-31 05:56:33',
   'tweet_id': 1355756838117986307,
   'urls': [],
   'medias': []},
  '1355756893684203522': {'text': 'Test 3\nThread part 2',
   'date': '2021-01-31 05:56:46',
   'tweet_id': 1355756893684203522,
   'urls': [],
   'medias': []},
  '1355756940379361287': {'text': 'Test 3\nThread part 3',
   'date': '2021-01-31 05:56:57',
   'tweet_id': 1355756940379361287,
   'urls': [],
   'medias': []},
  '1355756980879515649': {'text': 'Test 3\nThread part 4',
   'date': '2021-01-31 05:57:07',
   'tweet_id': 1355756980879515649,
   'urls': [],
   'medias': []},
  '1356171506833829891': {'text': 'Test 3\nOOps here is thread part 5',
   'date': '2021-02-01 09:24:18',
   'tweet_id': 1356171506833829891,
   'urls': [],
   'medias': []}

In [39]:
bottom_tracks = compiler.tweetCompilerBottom()
bottom_tracks

ThreadCompiler:Fetching Bottom thread!
ThreadCompiler: BottomThread - No more result found


{1356173077281992707: {'tweet': <__main__.Tweet at 0x22b31cf5d60>,
  'in_reply_to_status_id': 1355756940379361287},
 1356172678571438083: {'tweet': <__main__.Tweet at 0x22b31cf5880>,
  'in_reply_to_status_id': 1356172375952388096},
 1356172591858384897: {'tweet': <__main__.Tweet at 0x22b2f117cd0>,
  'in_reply_to_status_id': 1356171576580988928},
 1356172375952388096: {'tweet': <__main__.Tweet at 0x22b31ceafd0>,
  'in_reply_to_status_id': 1356172348072873987},
 1356172348072873987: {'tweet': <__main__.Tweet at 0x22b31ceab50>,
  'in_reply_to_status_id': 1356172290698997762},
 1356171757800062980: {'tweet': <__main__.Tweet at 0x22b31cea610>,
  'in_reply_to_status_id': 1355756893684203522},
 1356171576580988928: {'tweet': <__main__.Tweet at 0x22b31cea340>,
  'in_reply_to_status_id': 1356171506833829891},
 1356171506833829891: {'tweet': <__main__.Tweet at 0x22b31ceae80>,
  'in_reply_to_status_id': 1355756980879515649},
 1355758209420541954: {'tweet': <__main__.Tweet at 0x22b31cea670>,
  'in

In [40]:
for tweet in bottom_tracks:
    print(bottom_tracks[tweet]['tweet'].text)

@ThreaderTweet
random thread not of main thread 4/n
test 3
here is thread part 7
random thread not of main thread 3/n
random thread not of main thread 2/n
@ThreaderTweet
test 3
here is thread part 6
Test 3
OOps here is thread part 5
Some typo correction in test 3
Some more replies test thread part 3
Some random replies in between the thread test 3
Test 3
Thread part 4


In [71]:
compiler = ThreadCompiler(in_reply_to_status_id,in_reply_user_id,request_tweet_id)

In [72]:
tweets = compiler.compileThread()
tweets.to_dict()

ThreadCompiler: Compiling Thread
ThreadCompiler: Fetching Head thread!
ThreadCompiler:Fetching Bottom thread!
ThreadCompiler: BottomThread - No more result found
ThreadCompiler:Cooking Bottom Threads!
ThreadCompiler:Bottom Threads prepared!


{'id': 722743883617214464,
 'name': 'SobyDamn',
 'username': 'SobyDamn',
 'profile_img': 'https://pbs.twimg.com/profile_images/1328960664493285378/qRrzvNSN_normal.jpg',
 'tweets': {'1355531384589344771': {'text': 'Test 2\nTHis is thread part 1',
   'date': '2021-01-30 15:00:41',
   'tweet_id': 1355531384589344771,
   'urls': [],
   'medias': []},
  '1355531501132255242': {'text': 'Test 2\nTHis is thread part 2',
   'date': '2021-01-30 15:01:09',
   'tweet_id': 1355531501132255242,
   'urls': [],
   'medias': []},
  '1355531534346964997': {'text': 'Test 2\nTHis is thread part 3',
   'date': '2021-01-30 15:01:17',
   'tweet_id': 1355531534346964997,
   'urls': [],
   'medias': []},
  '1355531584355643397': {'text': 'Test 2\nTHis is thread part 4',
   'date': '2021-01-30 15:01:28',
   'tweet_id': 1355531584355643397,
   'urls': [],
   'medias': []},
  '1355531813704359936': {'text': 'Test 2\nTHis is thread part last',
   'date': '2021-01-30 15:02:23',
   'tweet_id': 1355531813704359936,
 

In [73]:
tweet = api.get_status(1355558754587189248, tweet_mode="extended")

In [95]:
tweet
medias = []
if hasattr(tweet,"extended_entities"):
    if "media" in tweet.extended_entities:
        media_entities = tweet.extended_entities['media']
        for media_data in media_entities:
            media = {}
            ##Currently only planning to save photo data as it'll be transformed to PDF
            if media_data['type']=='photo' or media_data['type'] == 'video':
                media['type'] = media_data['type']
                media['media_url_https'] = media_data['media_url_https']
                media['media_url'] = media_data['media_url']
                media['expanded_url'] = media_data['expanded_url']
                medias.append(media)

In [96]:
medias

[{'type': 'photo',
  'media_url_https': 'https://pbs.twimg.com/media/Es_pfkxWMAM3Q_c.png',
  'media_url': 'http://pbs.twimg.com/media/Es_pfkxWMAM3Q_c.png',
  'expanded_url': 'https://twitter.com/SobyDamn/status/1355558754587189248/photo/1'}]

In [126]:
bottom_thread = compiler.fetchBottomThread(tweets_track,tweet_id)
bottom_thread

ThreadCompiler:Cooking Bottom Threads!


In [129]:
for tweet in compiler.tweets:
    print(tweet.text)

Some random text
This is thread Part 1
This is thread Part 2
This is thread Part 3
This is thread Part 4
This is thread Part 6
This is thread Part 7


In [106]:
tweets_track

{1355196400557056001: {'tweet': <__main__.Tweet at 0x153ba764490>,
  'in_reply_to_status_id': 1355132897577955334},
 1355133016155197459: {'tweet': <__main__.Tweet at 0x153b999e3a0>,
  'in_reply_to_status_id': 1355132974098878465},
 1355132974098878465: {'tweet': <__main__.Tweet at 0x153b99dcf70>,
  'in_reply_to_status_id': 1355132940540252165},
 1355132940540252165: {'tweet': <__main__.Tweet at 0x153b7a32910>,
  'in_reply_to_status_id': 1355132897577955334}}

In [110]:
i=0
threads = {}
while i<len(list(tweets_track)):
    print(i,"i")
    looking_for = list(tweets_track)[i]
    child_id = looking_for
    i +=1
    count = 0
    possible_threads = []
    while looking_for:
        if looking_for in tweets_track:
            count +=1
            possible_threads.append(tweets_track[looking_for]['tweet'])
            print(looking_for)
            looking_for = tweets_track[looking_for]["in_reply_to_status_id"]
        elif looking_for == 1355132897577955334: #THread I was looking for
            threads[child_id] = possible_threads
            break
        else:
            print("End!")
            break
    print(count-1)
    i += count - 1

0 i
1355196400557056001
0
1 i
1355133016155197459
1355132974098878465
1355132940540252165
2


In [94]:
threads[1355144761066184714] = threads[1355133016155197459]
threads

{1355196400557056001: [<__main__.Tweet at 0x153ba6ccc70>],
 1355133016155197459: [<__main__.Tweet at 0x153ba6b0640>,
 1355144761066184714: [<__main__.Tweet at 0x153ba6b0640>,
  <__main__.Tweet at 0x153ba6b0880>]}

In [96]:
thread_id = max(threads, key=lambda k: len(threads[k]))
threads[thread_id][::-1]
thread_id

1355133016155197459

In [92]:
for tweet in threads[thread_id][::-1]:
    print(tweet.text)

This is thread Part 4
This is thread Part 6
This is thread Part 7


In [89]:
threads.values()


1355133016155197459

In [191]:
looking_for = in_reply_status_id
for tweet in list(timeline)[::-1]:
    print("Looking",tweet.text,looking_for)
    if hasattr(tweet, 'in_reply_to_status_id_str'):
        if tweet.in_reply_to_status_id == looking_for and tweet.id != tweet_id:
            looking_for = tweet.id
            print("found",tweet.text,tweet.id)

Looking This is thread Part 4 1355132897577955334
found This is thread Part 4 1355132940540252165
Looking This is thread Part 6 1355132940540252165
found This is thread Part 6 1355132974098878465
Looking This is thread Part 7 1355132974098878465
found This is thread Part 7 1355133016155197459


In [158]:
timeline[0]

Status(_api=<tweepy.api.API object at 0x000001C53EAF0460>, _json={'created_at': 'Fri Jan 29 12:37:24 +0000 2021', 'id': 1355132940540252165, 'id_str': '1355132940540252165', 'text': 'This is thread Part 4', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': 1355132897577955334, 'in_reply_to_status_id_str': '1355132897577955334', 'in_reply_to_user_id': 722743883617214464, 'in_reply_to_user_id_str': '722743883617214464', 'in_reply_to_screen_name': 'SobyDamn', 'user': {'id': 722743883617214464, 'id_str': '722743883617214464'}, 'geo': None, 'coordinates': None, 'place': None, 'contributors': None, 'is_quote_status': False, 'retweet_count': 0, 'favorite_count': 0, 'favorited': False, 'retweeted': False, 'lang': 'en'}, created_at=datetime.datetime(2021, 1, 29, 12, 37, 24), id=1355132940540252165, id_str='1355132940540252165'

In [165]:
for tweet in timeline[::-1]:
    print(tweet.text)

This is thread Part 4
This is thread Part 6
This is thread Part 7
@ThreaderTweet
Some.more tweets
Heya world
Dumbs


In [240]:
timelineTest = tweepy.Cursor(api.user_timeline,user_id=in_reply_user_id,trim_user=True,include_rts=False,exclude_replies=False,since_id=since_id,tweet_mode="extended").items()

In [242]:
tweets = []
for i in range(300):
    print(i)
    try:
        tweet = timelineTest.next()
        medias = []
        if 'media' in tweet.entities:
            media_entities = tweet.entities['media']
            for media_data in media_entities:
                ##Currently only planning to save photo data as it'll be transformed to PDF
                if media_data['type']=='photo':
                    medias.append(media_data['media_url_https'])
        tweetObj = Tweet(tweet.full_text,str(tweet.created_at),list(medias),tweet.id,tweet.entities['urls'])
        tweets.append(tweetObj)
        print(tweet.full_text)
    except StopIteration:
        print("Tweepy Cursor: No more result found")
        return True
    except tweepy.RateLimitError as e:
        logging.error("Twitter api rate limit reached".format(e))
        time.sleep(60)
        continue
    print("Am I getting triggered?")

SyntaxError: 'return' outside function (<ipython-input-242-fca8e1d9ef19>, line 18)

In [229]:
len(tweets)

300

In [231]:
with open("file.txt","w") as f:
    f.write(tweets)
    f.close()

TypeError: write() argument must be str, not list

In [232]:
track_tweets = {tweets[0].tweet_id:tweets[0]}

In [239]:
if tweets[0].tweet_id in track_tweets:
    print("Not exists")

Not exists


In [29]:
val = {123:{"a":1},
    1232:{"a":2},
    1223:{"a":1}
    }
keys = [x for x,y in val.items() if y['a'] == 1]
keys

[123, 1223]

In [245]:
a = [1,2,3,4]
b = [5,6,7,8]
a+b

[1, 2, 3, 4, 5, 6, 7, 8]

In [110]:
x = {1:[1,2,3,4],2:[1,2,3,4],3:[0]}
len(x)

3

In [18]:
def positiveNum(num):
    if num <0:
        raise Exception("Negative number")
    else:
        print("YEs passed")
    return num
def test():
    try:
        for i in range(10):
            print(positiveNum(i-5))
    except Exception as e:
        print("error",e)
test()

error Negative number


9
9
9
9
9
9
9
9
9
9
